# MAT281

## Aplicaciones de la Matemática en la Ingeniería

### Tarea - Módulos 3 y 4 

## Parte 1 - Versión inicial de visualización tipo Gapminder [30 puntos]

La siguiente tarea debe realizarse de manera individual. No es aceptable **bajo ninguna circunstancia** copiar y pegar código de otros estudiantes. Pueden consultar referencias en internet (stackoverflow, por ejemplo), pero deben indicar sus fuentes cuando corresponda y evitar copiar y pegar texto y código.
Todo el código debe ser ejecutable y las fórmulas en markdown y latex.

Enviar los archivos correspondientes a las partes 1, 2, 3 y 4 comprimidos en un zip, por correo.

**Nombre**: *[COLOCAR NOMBRE ACA]*

**Rol**: *[COLOCAR ROL ACA]*


**Fecha de entrega**: Martes 18 Diciembre 2018.

Esta tarea puede completarse en `jupyter lab` o `jupyter notebook` y requiere usar varias librerías: no dejes la instalación de las librerías para el último momento. Es aconsejable utilizar el ambiente `mat281_modulo4_tarea`.

El ambiente puede generarse directamente en el terminal con los comandos haciendo:

```
conda create --name mat281_modulo4_tarea
source activate mat281_modulo4_tarea
conda install scikit-learn
conda install matplotlib
conda install pandas
conda install jupyter notebook
conda install jupyter lab
conda install jupyter ipywidgets
conda install -c conda-forge altair
```
La librería altair es opcional, pudiendo trabajar solo con matplotlib si lo desean.

O bien, utilizando el archivo `mat281_modulo4_tarea.yml` provisto.

```
conda env create -f mat281_modulo4_tarea.yml
```

En jupyter notebook no es necesario ninguna configuración adicional. En jupyter lab, es necesario además en la terminal instalar la siguiente extensión:
```
jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

**Test de librerías**:

El siguiente código debería funcionar sin problemas, haciendo utilización de las diversas librerías. Debería verse como la imagen a continuación:

![EjemploIPYWIDGET](images/ipywidget.png "Ejemplo de ipywidget")

Figura 1: Ejemplo de `ipywidgtet`.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(m, b):
    plt.figure(figsize=(8,8))
    x = np.linspace(-6, 6, num=100)
    plt.plot(x, m * x + b, "k", lw=2.0)
    plt.xlim(-5, 5)
    plt.ylim(-5, 5)
    plt.grid(True)
    plt.show()

interactive(f, m=(-2.0, 3.0), b=(-3, 4, 0.5))

# Introducción

En Febrero del año 2006, Hans Rosling dictó el "Ted Talk" *The best stats you've ever seen*. Si aún no lo has visto, invierte 20 minutos en un excelente ejemplo de cómo utilizar visualizaciones para dar vida a datos, contar una historio y cautivar una audiencia.

In [ ]:
# Alternativamente, ver https://www.ted.com/talks/hans_rosling_shows_the_best_stats_you_ve_ever_seen
# Video
from IPython.lib.display import YouTubeVideo
YouTubeVideo('hVimVzgtD6w', width=600, height=450)

Además de ser un presentador carismático, Hans Rosling hace un buen uso de elementos técnicos para analizar tendencias e ilustrar diversos fenómenos. En esta tarea, utilizaremos estos mismos datos, proporcionados por la fundación [gapminder](https://www.gapminder.org/):
* Dataset de gapminder, con ciertas características de los paises y cómo varían en el tiempo.
* Tasa de fertilidad por país y año, medido en cantidad promedio de hijos por mujer.

## Generando un gráfico interactivo

Buscaremos reproducir el gráfico gapminder utilizando únicamente python.

Nuestra primera tarea será cargar los datos. Para ello usaremos `pandas` y crearemos un DataFrame llamado `df_no_color` puesto que posteriormente le agregaremos el color con el cual identificar a cada continente.

In [ ]:
# Cargando los datos
import pandas as pd
import os

# Open the file
fpath = os.path.join("data","gapminderData.csv")
df_no_color = pd.read_csv(fpath, sep=",")

# Rename the columns
df_no_color = df_no_color.rename(columns={"pop":"population", "lifeExp":"life_expectancy", "gdpPercap":"gdp_per_capita"})

# Print the first lines
df_no_color.head()

Como es habitual, usemos el método `describe` para familiarizarnos con el contenido y las columnas:

In [ ]:
df_no_color.describe(include="all").fillna("").T

Necesitaremos identificar cada Continente por un color, por lo cual crearemos una relación entre el Continente y el color para representarlo. Esta relación la indicamos con un dataframe para el cual indicamos los datos de manera directa:

In [ ]:
# Add color
data = [["Asia","red"],["Europe","orange"],["Africa", "blue"],["Americas","yellow"],["Oceania","red"]]
df_color = pd.DataFrame(data, columns=["continent", "color"])
df_color

Por último, generaremos el dataframe a utilizar haciendo un merge entre el dataframe de datos sin color, y el dataframe de la relación continente-color:

In [ ]:
global df # Para poder usar el dataframe en funciones y ipywidget.
df = df_no_color.merge(df_color)

Antes de continuar, veamos lo que hemos obtenido:

In [ ]:
df.head()

La librería `interactive` es simplemente un procedimiento que toma una función para graficar (en nuestro caso, `gapminder_plot_basic`, y que le entrega los parámetros con los cuales debe generar un gráfico).

Debemos por tanto definir una función que grafique la relación que buscamos para un cierto año.

In [ ]:
# Libraries
import matplotlib.pyplot as plt

# Función de gráfico
def gapminder_plot_basic(year):
    """
    Plotting function: giving a year, plots (gdp, life_expectancy) with circle
    proportional to population.
    """
    # Data wrangling
    m_year = df.year==year
    x = df[m_year].gdp_per_capita.values
    y = df[m_year].life_expectancy.values
    z = df[m_year].population.values
    s = 5000*z/z.max()
    c = df[m_year].color.values
    # Graph
    plt.figure(figsize=(8,8))
    plt.title("Year {}".format(year))
    plt.scatter(x, y, s, c=c, alpha=0.25)
    plt.show()

# Prueba
year = 1982
gapminder_plot_basic(year)    

La versión básica de animación simplemente utiliza los distintos valores de los años:

In [ ]:
# Load the graph
year_values = sorted(df.year.unique())
interactive(gapminder_plot_basic, year=year_values)

Lo anterior no es satisfactorio porque no permite "jugar" con los años de manera natural. El ideal sería tener una slider que nos permitiera avanzar naturalmente por los años.

In [ ]:
# Libraries
from ipywidgets import interactive
from ipywidgets import IntSlider
# Load the graph
year_values = IntSlider(min=1952, max=2007, step=5)
interactive(gapminder_plot_basic, year=year_values)

Lo anterior ya es bastante parecido al gráfico de gapminder, pero existen varios defectos que tenemos que trabajar:
* Los ejes $x$ e $y$ no tienen unidades.
* Los ejes $x$ e $y$ cambian su rango de valores según el año. El rango debería estar fijo para poder observar mejor las tendencias y fluctuaciones.
* El gráfico no contiene la codificación de colores usada para los continentes, por lo que no se entienden las tendencias.
* Existe demasiada información, que no puede analizarse simultáneamente. Debería poder seleccionarse continentes.
* No es posible identificar cada país. Deberíamos poder poner algunos textos de manera selectiva.

En función de lo anterior, generamos una nueva función `gapminder_plot` que es más genérico y profesional. En la misma celda definimos la función y hacemos la lanzamos interactivamente:

In [ ]:
# Libraries
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from ipywidgets import interactive
from ipywidgets import IntSlider

# Create the 
def gapminder_plot(year, continent, country_1, country_2):
    """
    A humble interactive python implementation of gapminder.
    """
    # Get maximum and minimum values - for any year
    x_min = df.gdp_per_capita.min()
    x_max = df.gdp_per_capita.max()
    y_min = df.life_expectancy.min()
    y_max = df.life_expectancy.max()
    z_max = df.population.values.max()
    # Data wrangling - continent and year masks
    m_year = df.year==year
    if continent=="All":
        m_continent = df.continent!="" # All True
    elif continent=="Selected countries":
        m_continent = pd.np.logical_or(df.country==country_1, df.country==country_2)
    else:
        m_continent = df.continent==continent
    m_all = pd.np.logical_and(m_year, m_continent)
    # Data wrangling
    x = df[m_all].gdp_per_capita.values
    y = df[m_all].life_expectancy.values
    z = df[m_all].population.values
    sizes = 8000*z/z_max
    colors = df[m_all].color.values
    countries = df[m_all].country.values
    # Graph
    plt.figure(figsize=(8,8))
    plt.scatter(x, y, sizes, c=colors, alpha=0.3)
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    # Country names - country 1
    m_1 = df.country==country_1
    m_all_1 = pd.np.logical_and(m_1, m_all)
    if m_all_1.sum() == 1:
        x_1 = df[m_all_1].gdp_per_capita
        y_1 = df[m_all_1].life_expectancy
        plt.text(x_1, y_1, country_1, alpha=0.8)
    # Country names - country 2
    m_2 = df.country==country_2
    m_all_2 = pd.np.logical_and(m_2, m_all)
    if m_all_2.sum() == 1:
        x_2 = df[m_all_2].gdp_per_capita
        y_2 = df[m_all_2].life_expectancy
        plt.text(x_2, y_2, country_2, alpha=0.8)
    # Custom legend
    df_continents = df[["color","continent"]].drop_duplicates()
    patches = []
    for color, continent in df_continents.values:
        patches.append(mpatches.Patch(color=color, label=continent, alpha=0.3))
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    # Axis
    plt.xlabel("GDP per capita [US dollars]")
    plt.ylabel("Life Expectancy [years]")
    plt.title("Year {} - Circle size proportional to population".format(year))
    # Show
    plt.show()


# Load the graph
year_values = IntSlider(min=1952,max=2007,step=5)
continent_dict = ["All","Selected countries", "Asia", "Europe", "Africa", "Americas", "Oceania"]
countries = ["- None -"] + sorted(list(df.country.unique()))
interactive(gapminder_plot, year=year_values, continent=continent_dict, country_1=countries, country_2=countries)

El gráfico anterior permite:
* Seleccionar un continente en específico.
* Colocar el nombre de 2 países, y seleccionar únicamente esos países.
* Variar el año en análisis.

Toma 5 minutos revisar el código anterior  para considerar que hemos obtenido un gráfico que parece bastante avanzado y sin embargo, al mirar el código, es bastante sencillo.

## Análisis del gráfico generado

### Pregunta 1.1 [10 puntos]
Elabore una lista de las distintas variables utilizadas en este gráfico, indicando ejemplos de valores, clasificación (tipo) de dato y representación gráfica utilizada. Sería por tanto relevante recordar [el contenido de la clase 2 de visualización](https://github.com/sebastiandres/mat281_m03_visualizacion/blob/master/02_teoria_visualizacion/02_teoria_visualizacion.ipynb).

*R:* 
* (1.) Nombre dato - ejemplo valores - tipo dato - representación grafica utilizada. [#FIX ME#]

...

* (N.) Nombre dato - ejemplo valores - tipo dato - representación grafica utilizada.  [#FIX ME#]

Por ejemplo, si el gráfico tuviera información sobre el peso promedio de cada persona en el país, una de las líneas anteriores podría haber sido 

*"Peso promedio per cápita - 78.2 [kg] - dato cuantitativo - radio de circulo utilizado para cada país".*


###  Pregunta 1.2 [10 puntos]
¿Para que país y año se tienen los 5 mayores valores de gdp per capita?

Completar el código a continuación para obtener la respuesta. Una respuesta podría utilizar los métodos `sort_values` y `head`.

In [ ]:
df#FIX ME

*R:* En el año XYZ el país ZYX tiene un gdp per capita de X...

###  Pregunta 1.3 [10 puntos]

¿Cómo ha ido cambiando, año a año, el promedio de esperanza de vida por continente? La respuesta requiere usar los métodos `groupby` y `pivot_table`.

In [ ]:
dfg = df#.groupby().agg() # FIX ME #
dfg#.pivot_table() # FIX ME # 

*R:* A partir de la tabla anterior, vemos que ...